In [ ]:
#updated query to identify the first bp reading within 12 months before care home admission, uses updated social care data
CREATE OR REPLACE TABLE `yhcr-prd-bradfor-bia-core.CB_2172.cb_2172_bp_pre_admission` as (
#selects all bp readings for the care home cohort in those with a diagnosis of hypertension before admission date
with primary_care_code as (
  select
  person_id,
  ctv3code,
  ctv3text,
  dateevent,
  numericvalue
  from `CB_FDM_PrimaryCare.tbl_srcode`
  where ctv3code in(SELECT CTV3_CONCEPTID FROM `CB_2172.bloodpressure_codes_snomed_codes_rough`) and person_id in(select person_id from `yhcr-prd-bradfor-bia-core.CB_2172.cb_2172_hypertension_combined`)
),

hypertension_ch as (
  select
  person_id,
  first_episodestartdate,
  hypertension
  from
   `CB_2172.cb_2172_hypertension_combined`
),
 bp_primary_care as (
SELECT 
a.person_id, 
b.ctv3code, 
b.ctv3text,
CAST(b.numericvalue as bignumeric) bp_value,
b.dateevent,
a.first_episodestartdate
FROM hypertension_ch a
INNEr JOIN primary_care_code b
ON a.person_id = b.person_id
WHERE (a.hypertension = true) and (b.dateevent
< a.first_episodestartdate)),
# filter out those with a value less than 10 as these will be not be true values, duplicate_count, 
bp_primary_care_filtered as (
select
*,
row_number() over (partition by bp_primary_care.bp_value, dateevent, person_id) as dup_count
 from bp_primary_care
 where bp_value > 10),
# remove duplicates
bp_readings_filtered_no_duplicates as (
select
* 
from bp_primary_care_filtered
where dup_count=1),

diastolic as (
    SELECT
  *, 
  bp_value as diastolic_value,
  FROM bp_readings_filtered_no_duplicates
  WHERE ctv3text like "%iastolic%"),

  systolic as (
    SELECT 
    *,
    bp_value as systolic_value
    FROM bp_readings_filtered_no_duplicates
    WHERE ctv3text like "%ystolic%"),

reading_join as (
  SELECT 
  diastolic.person_id,
  systolic_value,
  diastolic_value,
  diastolic.dateevent as datetimeevent_diastolic,
  systolic.dateevent as datetimeevent_systolic,
   diastolic.first_episodestartdate,
  extract(date from systolic.dateevent) as date_systolic,
  FROM diastolic
  FULL JOIN systolic 
  ON 
  diastolic.person_id = systolic.person_id AND diastolic.dateevent = systolic.dateevent
  #here i have filtered out reading which, were they a true measurement of a person's blood pressure, would result in urgnet hospital admission or indicate someone who is dying. these values which chosen empirically based on my own clinical knowledge. 
  WHERE datetime_diff(diastolic.first_episodestartdate,diastolic.dateevent,day) < 366 AND systolic_value >79 AND systolic_value <230 AND diastolic_value >49),

  filter_daily as (
#here i have selected the first blood pressure recording following admission. 
  SELECT
  *,
  rank() OVER (PARTITION BY person_id ORDER BY datetimeevent_systolic) as bp_seq,
  row_number() over (partition by person_id, date_systolic order by systolic_value) as systolic_daily
  from 
  reading_join rj)

select
* 
from filter_daily
where systolic_daily = 1 and bp_seq = 1
order by person_id)
